In [1]:
import pandas as pd
import numpy as np

In [3]:
movies_df = pd.read_csv('movies_sample.csv',usecols=['movieId','title'],dtype={'movieId': 'int32', 'title': 'str'})

In [5]:
rating_df = pd.read_csv('ratings.csv',usecols=['userId', 'movieId', 'rating'],
    dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [7]:
movies_df.head()

,movieId,title
0,1,Toy Story (1995)
1,2,Jumanji (1995)
2,3,Grumpier Old Men (1995)
3,4,Waiting to Exhale (1995)
4,5,Father of the Bride Part II (1995)


In [9]:
rating_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [11]:
## merge two dataset considering movieid
df = pd.merge(rating_df,movies_df , on='movieId')
df.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,1,3,4.0,Grumpier Old Men (1995)
2,1,6,4.0,Heat (1995)
3,1,47,5.0,Seven (a.k.a. Se7en) (1995)
4,1,50,5.0,"Usual Suspects, The (1995)"


In [19]:
##count how many person rating a movie

count_movie_rating = df.dropna(axis=0 ,subset = ['title' ])  ###missing nan values



movie_ratingcount = (count_movie_rating.
                     groupby(by = ['title'])['rating'].
                     count().
                     reset_index().
                     rename(columns = {'rating':'totalRatingCount'})
                     [['title','totalRatingCount']]
                     
                    )
movie_ratingcount.head()

,title,totalRatingCount
0,'71 (2014),1
1,'Hellboy': The Seeds of Creation (2004),1
2,'Round Midnight (1986),2
3,'Salem's Lot (2004),1
4,'Til There Was You (1997),2


In [23]:
rating_with_totalRatingCount = count_movie_rating.merge(movie_ratingcount , left_on = 'title' , right_on = 'title' , how='left')

rating_with_totalRatingCount.head()


,userId,movieId,rating,title,totalRatingCount
0,1,1,4.0,Toy Story (1995),215
1,1,3,4.0,Grumpier Old Men (1995),52
2,1,6,4.0,Heat (1995),102
3,1,47,5.0,Seven (a.k.a. Se7en) (1995),203
4,1,50,5.0,"Usual Suspects, The (1995)",204


In [27]:
print(movie_ratingcount['totalRatingCount'].describe())

count    9719.000000
mean       10.375141
std        22.406220
min         1.000000
25%         1.000000
50%         3.000000
75%         9.000000
max       329.000000
Name: totalRatingCount, dtype: float64


In [29]:
##popularity count
popularity_threshold = 50

rating_popular_movie = rating_with_totalRatingCount.query('totalRatingCount > = @popularity_threshold')
rating_popular_movie.head()

,userId,movieId,rating,title,totalRatingCount
0,1,1,4.0,Toy Story (1995),215
1,1,3,4.0,Grumpier Old Men (1995),52
2,1,6,4.0,Heat (1995),102
3,1,47,5.0,Seven (a.k.a. Se7en) (1995),203
4,1,50,5.0,"Usual Suspects, The (1995)",204


In [33]:
###creat a pivot table
movie_features_df = rating_popular_movie.pivot_table(index='title',columns='userId',values='rating').fillna(0)
movie_features_df.head()


userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0


In [39]:
from scipy.sparse import csr_matrix
## non-zero elements and their row/column indices.
######  (1, 2) 3      means the value at row 1, column 2 is 3.
#######
3#######
movie_features_df_matrix = csr_matrix(movie_features_df.values)

from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_features_df_matrix)


NearestNeighbors(algorithm='brute', metric='cosine')

In [41]:
movie_features_df_matrix

<450x606 sparse matrix of type '<class 'numpy.float32'>'
	with 41360 stored elements in Compressed Sparse Row format>

In [43]:
movie_features_df_matrix.shape

(450, 606)

In [53]:
## movie randomly selected

query_index = np.random.choice(movie_features_df.shape[0])
print(query_index)

138


In [55]:
distances,indices = model_knn.kneighbors(movie_features_df.iloc[query_index,:].values.reshape(1,-1),n_neighbors=6)


In [63]:
###distances.flatten(): This flattens the 2D array of distances into a 1D array, 

for i in range(0 ,len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(movie_features_df.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, movie_features_df.index[indices.flatten()[i]], distances.flatten()[i]))

                       

Recommendations for Ed Wood (1994):

1: Get Shorty (1995), with distance of 0.510016918182373:
2: Quiz Show (1994), with distance of 0.551247239112854:
3: Four Weddings and a Funeral (1994), with distance of 0.558853268623352:
4: Clerks (1994), with distance of 0.5813702344894409:
5: Dave (1993), with distance of 0.5901564359664917:
